In [4]:
# Import initial dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from xgboost.sklearn import XGBClassifier  
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
%matplotlib inline

In [5]:
# Read in master wine table and conver to DF
df = pd.read_csv("../data/master_wine_table.csv")

In [6]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,freq,c_freq,red,rose,white,sparkling,flavor_categories_taste_notes
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2466.0,4856.0,1.0,0.0,0.0,0.0,"light, fruity"
1,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,13272.0,52329.0,1.0,0.0,0.0,0.0,"medium bodied, balanced"
2,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,9472.0,52329.0,1.0,0.0,0.0,0.0,"light, fruity"
3,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini,9472.0,52329.0,1.0,0.0,0.0,0.0,"medium bodied, balanced"
4,Argentina,"Baked plum, molasses, balsamic vinegar and che...",Felix,87,30.0,Other,Cafayate,NaN,Michael Schachner,Felix Lavaque 2010 Felix Malbec (Cafayate),Malbec,Felix Lavaque,2652.0,3502.0,1.0,0.0,0.0,0.0,"medium bodied, balanced"


In [7]:
 df.columns[df.isna().any()].tolist()

['designation', 'region_1', 'region_2', 'taster_name']

In [8]:
len(df)

109662

In [9]:
df_no_nans = df.dropna(subset = ['taster_name'])
len(df_no_nans)

86944

In [10]:
# Create feature DF
feature_df = df_no_nans[['points','country','price', 'red', 'rose', 'white', 'sparkling',
       'taster_name']]

In [11]:
# Encode the target for the train and test set so it can be fed into our NN and conver to a vector and check it
data = feature_df.copy()

data_binary_encoded = pd.get_dummies(data, columns=["country", "taster_name"])
data_binary_encoded.head()

,points,price,red,rose,white,sparkling,country_Argentina,country_Australia,country_Austria,country_Brazil,...,taster_name_Kerin O’Keefe,taster_name_Lauren Buzzeo,taster_name_Matt Kettmann,taster_name_Michael Schachner,taster_name_Mike DeSimone,taster_name_Paul Gregutt,taster_name_Roger Voss,taster_name_Sean P. Sullivan,taster_name_Susan Kostrzewa,taster_name_Virginie Boone
0,87,15.0,1.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,87,65.0,1.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,87,19.0,1.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,87,34.0,1.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,87,30.0,1.0,0.0,0.0,0.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
data_binary_encoded.dtypes

points                              int64
price                             float64
red                               float64
rose                              float64
white                             float64
sparkling                         float64
country_Argentina                   uint8
country_Australia                   uint8
country_Austria                     uint8
country_Brazil                      uint8
country_Bulgaria                    uint8
country_Canada                      uint8
country_Chile                       uint8
country_England                     uint8
country_France                      uint8
country_Germany                     uint8
country_Greece                      uint8
country_Hungary                     uint8
country_Israel                      uint8
country_Italy                       uint8
country_Lebanon                     uint8
country_Mexico                      uint8
country_Moldova                     uint8
country_Morocco                   

In [13]:
data_binary_encoded.columns

Index(['points', 'price', 'red', 'rose', 'white', 'sparkling',
       'country_Argentina', 'country_Australia', 'country_Austria',
       'country_Brazil', 'country_Bulgaria', 'country_Canada', 'country_Chile',
       'country_England', 'country_France', 'country_Germany',
       'country_Greece', 'country_Hungary', 'country_Israel', 'country_Italy',
       'country_Lebanon', 'country_Mexico', 'country_Moldova',
       'country_Morocco', 'country_New Zealand', 'country_Portugal',
       'country_Romania', 'country_Slovenia', 'country_South Africa',
       'country_Spain', 'country_Turkey', 'country_US', 'country_Uruguay',
       'taster_name_Alexander Peartree', 'taster_name_Anna Lee C. Iijima',
       'taster_name_Anne Krebiehl MW', 'taster_name_Carrie Dykes',
       'taster_name_Christina Pickard', 'taster_name_Fiona Adams',
       'taster_name_Jeff Jenssen', 'taster_name_Jim Gordon',
       'taster_name_Joe Czerwinski', 'taster_name_Kerin O’Keefe',
       'taster_name_Lauren Buzzeo'

In [14]:
data_binary_encoded2 = data_binary_encoded.rename({'taster_name_Anne Krebiehl MW': 'taster_name_Anne Krebiehl'}, axis=1)

In [15]:
data_binary_encoded2.columns

Index(['points', 'price', 'red', 'rose', 'white', 'sparkling',
       'country_Argentina', 'country_Australia', 'country_Austria',
       'country_Brazil', 'country_Bulgaria', 'country_Canada', 'country_Chile',
       'country_England', 'country_France', 'country_Germany',
       'country_Greece', 'country_Hungary', 'country_Israel', 'country_Italy',
       'country_Lebanon', 'country_Mexico', 'country_Moldova',
       'country_Morocco', 'country_New Zealand', 'country_Portugal',
       'country_Romania', 'country_Slovenia', 'country_South Africa',
       'country_Spain', 'country_Turkey', 'country_US', 'country_Uruguay',
       'taster_name_Alexander Peartree', 'taster_name_Anna Lee C. Iijima',
       'taster_name_Anne Krebiehl MW', 'taster_name_Carrie Dykes',
       'taster_name_Christina Pickard', 'taster_name_Fiona Adams',
       'taster_name_Jeff Jenssen', 'taster_name_Jim Gordon',
       'taster_name_Joe Czerwinski', 'taster_name_Kerin O’Keefe',
       'taster_name_Lauren Buzzeo'

In [17]:
X = data_binary_encoded[['price', 'red', 'rose', 'white', 'sparkling','country_Argentina', 
        'country_Australia', 'country_Austria',
       'country_Brazil', 'country_Bulgaria', 'country_Canada', 'country_Chile',
       'country_England', 'country_France', 'country_Germany',
       'country_Greece', 'country_Hungary', 'country_Israel', 'country_Italy',
       'country_Lebanon', 'country_Mexico', 'country_Moldova',
       'country_Morocco', 'country_New Zealand', 'country_Portugal',
       'country_Romania', 'country_Slovenia', 'country_South Africa',
       'country_Spain', 'country_Turkey', 'country_US', 'country_Uruguay',
       'taster_name_Alexander Peartree', 'taster_name_Anna Lee C. Iijima',
        'taster_name_Carrie Dykes',
       'taster_name_Christina Pickard', 'taster_name_Fiona Adams',
       'taster_name_Jeff Jenssen', 'taster_name_Jim Gordon',
       'taster_name_Joe Czerwinski', 'taster_name_Kerin O’Keefe',
       'taster_name_Lauren Buzzeo', 'taster_name_Matt Kettmann',
       'taster_name_Michael Schachner', 'taster_name_Mike DeSimone',
       'taster_name_Paul Gregutt', 'taster_name_Roger Voss',
       'taster_name_Sean P. Sullivan', 'taster_name_Susan Kostrzewa',
       'taster_name_Virginie Boone']]
Y = data_binary_encoded['points']

In [18]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 88.37364139790654
Coefficients: 
 [ 0.02439491  0.49719716 -0.78797631  0.07343944  0.21733972 -0.41811582
  1.22216015  1.61680603 -2.34859929 -0.83304913 -0.04248384 -0.49721471
  1.70813426  1.07914278  1.60708857  1.66432382 -0.32623324  0.44243144
  0.70629421 -0.77474124 -2.0010372  -1.28197307 -0.14188855  1.15627859
  1.0524952  -2.0977512  -1.0761236   2.02356111  0.11606912 -1.01249298
 -0.70862934 -0.83445208 -2.79101239 -0.95007547 -2.22038383 -1.44010284
 -2.05435881  0.1875443  -0.05060261 -2.09749998 -1.59118585 -3.0730946
  1.08423439 -2.25697279 -0.26973944  0.23009715 -2.05589309 -0.11073759
 -3.84408642  0.05320772]


In [20]:
coeff_df = pd.DataFrame(regr.coef_, X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
price,0.024395
red,0.497197
rose,-0.787976
white,0.073439
sparkling,0.217340
country_Argentina,-0.418116
country_Australia,1.222160
country_Austria,1.616806
country_Brazil,-2.348599
country_Bulgaria,-0.833049
